In [2]:
import pandas as pd 



In [4]:
One_participant = pd.read_csv("/Volumes/SSD/00_Data_Processing/Pre_processed/04_Interpolated/1754_2.csv" )

In [5]:
######## Debbies Algorithm########

for_eye = One_participant.copy()
time = for_eye["timeStampDataPointEnd"].tolist()
# get individual coordinates
subj = list(zip( for_eye["eyePositionCombinedWorld.x"],for_eye["eyePositionCombinedWorld.y"],for_eye["eyePositionCombinedWorld.z"]))
hpoo = list(zip(for_eye["hitPointOnObject_x"], for_eye["hitPointOnObject_y"],for_eye["hitPointOnObject_z"]))

# v_gaze_vec: get difference in hpoo
v_vX = pd.DataFrame(for_eye["hitPointOnObject_x"].diff().tolist())
v_vY = pd.DataFrame(for_eye["hitPointOnObject_y"].diff().tolist())
v_vZ = pd.DataFrame(for_eye["hitPointOnObject_z"].diff().tolist())

In [13]:
for_eye["hitPointOnObject_x"].diff().tolist()[59]

0.02867889404296875

In [5]:

hpoo = list(zip(for_eye["hitPointOnObject_x"], for_eye["hitPointOnObject_y"],for_eye["hitPointOnObject_z"]))

In [14]:
v_vX[59]

0.02867889404296875

In [9]:
v_vX

[nan,
 -0.045772552490234375,
 -0.00293731689453125,
 -0.005062103271484375,
 -0.055507659912109375,
 0.032478332519524145,
 -0.02892303466795454,
 0.01511001586913352,
 0.005359649658203125,
 0.060420989990234375,
 0.037342071533203125,
 -0.0200042724609375,
 0.000255584716796875,
 -0.017459869384765625,
 7.62939453125e-05,
 0.1027221679687429,
 -0.02846145629882102,
 -0.09725189208984375,
 -0.16277313232421875,
 -0.07417678833008523,
 -0.0026054382324147696,
 0.012451171875,
 0.02596282958984375,
 -0.13100051879882812,
 -0.12306594848632812,
 0.000629425048828125,
 0.02070236206053977,
 -0.001743316650390625,
 0.0967407226562571,
 -0.00687408447265625,
 0.08967208862304688,
 -0.04429244995117898,
 -0.005226135253899145,
 0.0394744873046875,
 -0.0294036865234375,
 0.004451751708984375,
 -0.042507171630859375,
 0.09400558471679688,
 0.00928497314453125,
 0.01031494140625,
 0.01197052001953125,
 0.5524864196777273,
 0.9139099121093821,
 0.0464935302734375,
 0.033893585205078125,
 -0.669

In [2]:
path = "/Volumes/SSD/6258_PointingTask_Randomization_1666084297.70049.json"
with open(path, 'r') as file:
        # make json files parsable
        data = "[" + file.read()
        data = data[:len(data)-1] + "]"

        # read data per file
        subjectdf = pd.read_json(data)

        # insert participant id in every line
        subjectdf.insert(0, "SubjectID", [int(path[13:17])] * subjectdf.shape[0], True)

In [3]:
subjectdf.head()

,SubjectID,ParticipantID,TaskOrder,Language,InitialOrientation,PointingTaskRandomised,PointingTaskStartingLocations,PointingTaskTargets_1,PointingTaskTargets_2,PointingTaskTargets_3,...,PointingTaskTargets_19,PointingTaskTargets_20,PointingTaskTargets_21,PointingTaskTargets_22,PointingTaskTargets_23,PointingTaskTargets_24,PointingTaskTargets_25,PointingTaskTargets_26,PointingTaskTargets_27,PointingTaskTargets_28
0,6258,6258,[PointingTask],English,"{'x': 0.0008186895865940001, 'y': -0.906321287...","[{'Block': 1, 'LocationNumber': 1, 'StartingPo...","[4, 25, 27, 13, 26, 6, 11, 23, 21, 19, 24, 10,...","[28, 43, 99, 99, 107, 22, 92, 41]","[9, 85, 27, 32, 25, 31, 14, 2]","[54, 93, 10, 36, 34, 54, 34, 28]",...,"[77, 24, 75, 107, 50, 64, 23, 36]","[44, 56, 89, 6, 97, 37, 97, 85]","[100, 13, 13, 87, 38, 54, 46, 62]","[104, 71, 8, 104, 22, 25, 52, 54]","[27, 73, 48, 0, 93, 89, 44, 58]","[67, 77, 0, 24, 52, 56, 55, 13]","[19, 101, 18, 46, 31, 23, 89, 12]","[2, 90, 55, 31, 13, 66, 30, 63]","[69, 76, 85, 80, 83, 64, 70, 71]","[81, 76, 65, 60, 75, 77, 77, 63]"


In [4]:
StartingPoints = pd.DataFrame(subjectdf.explode('PointingTaskStartingLocations'))
StartingPoints.head()

,SubjectID,ParticipantID,TaskOrder,Language,InitialOrientation,PointingTaskRandomised,PointingTaskStartingLocations,PointingTaskTargets_1,PointingTaskTargets_2,PointingTaskTargets_3,...,PointingTaskTargets_19,PointingTaskTargets_20,PointingTaskTargets_21,PointingTaskTargets_22,PointingTaskTargets_23,PointingTaskTargets_24,PointingTaskTargets_25,PointingTaskTargets_26,PointingTaskTargets_27,PointingTaskTargets_28
0,6258,6258,[PointingTask],English,"{'x': 0.0008186895865940001, 'y': -0.906321287...","[{'Block': 1, 'LocationNumber': 1, 'StartingPo...",4,"[28, 43, 99, 99, 107, 22, 92, 41]","[9, 85, 27, 32, 25, 31, 14, 2]","[54, 93, 10, 36, 34, 54, 34, 28]",...,"[77, 24, 75, 107, 50, 64, 23, 36]","[44, 56, 89, 6, 97, 37, 97, 85]","[100, 13, 13, 87, 38, 54, 46, 62]","[104, 71, 8, 104, 22, 25, 52, 54]","[27, 73, 48, 0, 93, 89, 44, 58]","[67, 77, 0, 24, 52, 56, 55, 13]","[19, 101, 18, 46, 31, 23, 89, 12]","[2, 90, 55, 31, 13, 66, 30, 63]","[69, 76, 85, 80, 83, 64, 70, 71]","[81, 76, 65, 60, 75, 77, 77, 63]"
0,6258,6258,[PointingTask],English,"{'x': 0.0008186895865940001, 'y': -0.906321287...","[{'Block': 1, 'LocationNumber': 1, 'StartingPo...",25,"[28, 43, 99, 99, 107, 22, 92, 41]","[9, 85, 27, 32, 25, 31, 14, 2]","[54, 93, 10, 36, 34, 54, 34, 28]",...,"[77, 24, 75, 107, 50, 64, 23, 36]","[44, 56, 89, 6, 97, 37, 97, 85]","[100, 13, 13, 87, 38, 54, 46, 62]","[104, 71, 8, 104, 22, 25, 52, 54]","[27, 73, 48, 0, 93, 89, 44, 58]","[67, 77, 0, 24, 52, 56, 55, 13]","[19, 101, 18, 46, 31, 23, 89, 12]","[2, 90, 55, 31, 13, 66, 30, 63]","[69, 76, 85, 80, 83, 64, 70, 71]","[81, 76, 65, 60, 75, 77, 77, 63]"
0,6258,6258,[PointingTask],English,"{'x': 0.0008186895865940001, 'y': -0.906321287...","[{'Block': 1, 'LocationNumber': 1, 'StartingPo...",27,"[28, 43, 99, 99, 107, 22, 92, 41]","[9, 85, 27, 32, 25, 31, 14, 2]","[54, 93, 10, 36, 34, 54, 34, 28]",...,"[77, 24, 75, 107, 50, 64, 23, 36]","[44, 56, 89, 6, 97, 37, 97, 85]","[100, 13, 13, 87, 38, 54, 46, 62]","[104, 71, 8, 104, 22, 25, 52, 54]","[27, 73, 48, 0, 93, 89, 44, 58]","[67, 77, 0, 24, 52, 56, 55, 13]","[19, 101, 18, 46, 31, 23, 89, 12]","[2, 90, 55, 31, 13, 66, 30, 63]","[69, 76, 85, 80, 83, 64, 70, 71]","[81, 76, 65, 60, 75, 77, 77, 63]"
0,6258,6258,[PointingTask],English,"{'x': 0.0008186895865940001, 'y': -0.906321287...","[{'Block': 1, 'LocationNumber': 1, 'StartingPo...",13,"[28, 43, 99, 99, 107, 22, 92, 41]","[9, 85, 27, 32, 25, 31, 14, 2]","[54, 93, 10, 36, 34, 54, 34, 28]",...,"[77, 24, 75, 107, 50, 64, 23, 36]","[44, 56, 89, 6, 97, 37, 97, 85]","[100, 13, 13, 87, 38, 54, 46, 62]","[104, 71, 8, 104, 22, 25, 52, 54]","[27, 73, 48, 0, 93, 89, 44, 58]","[67, 77, 0, 24, 52, 56, 55, 13]","[19, 101, 18, 46, 31, 23, 89, 12]","[2, 90, 55, 31, 13, 66, 30, 63]","[69, 76, 85, 80, 83, 64, 70, 71]","[81, 76, 65, 60, 75, 77, 77, 63]"
0,6258,6258,[PointingTask],English,"{'x': 0.0008186895865940001, 'y': -0.906321287...","[{'Block': 1, 'LocationNumber': 1, 'StartingPo...",26,"[28, 43, 99, 99, 107, 22, 92, 41]","[9, 85, 27, 32, 25, 31, 14, 2]","[54, 93, 10, 36, 34, 54, 34, 28]",...,"[77, 24, 75, 107, 50, 64, 23, 36]","[44, 56, 89, 6, 97, 37, 97, 85]","[100, 13, 13, 87, 38, 54, 46, 62]","[104, 71, 8, 104, 22, 25, 52, 54]","[27, 73, 48, 0, 93, 89, 44, 58]","[67, 77, 0, 24, 52, 56, 55, 13]","[19, 101, 18, 46, 31, 23, 89, 12]","[2, 90, 55, 31, 13, 66, 30, 63]","[69, 76, 85, 80, 83, 64, 70, 71]","[81, 76, 65, 60, 75, 77, 77, 63]"


In [5]:
StartingPointsnorm = pd.json_normalize(subjectdf["InitialOrientation"]).add_prefix("InitialOrientation_")
StartingPointsnorm.head()

,InitialOrientation_x,InitialOrientation_y,InitialOrientation_z,InitialOrientation_w
0,0.000819,-0.906321,-0.001439,-0.422586


In [6]:
%timeit -r 4 -n 1000 StartingPoints = pd.DataFrame(subjectdf.explode('PointingTaskStartingLocations'))
%timeit -r 4 -n 1000 StartingPointsnorm = pd.json_normalize(subjectdf["InitialOrientation"])

3.12 ms ± 70.9 µs per loop (mean ± std. dev. of 4 runs, 1000 loops each)
520 µs ± 12.8 µs per loop (mean ± std. dev. of 4 runs, 1000 loops each)


In [7]:
subjectdf["PointingTaskRandomised"][0]

[{'Block': 1,
  'LocationNumber': 1,
  'StartingPositionRandom': 4,
  'TargetLocationsRandom': [28, 43, 99, 99, 107, 22, 92, 41]},
 {'Block': 1,
  'LocationNumber': 2,
  'StartingPositionRandom': 25,
  'TargetLocationsRandom': [9, 85, 27, 32, 25, 31, 14, 2]},
 {'Block': 1,
  'LocationNumber': 3,
  'StartingPositionRandom': 27,
  'TargetLocationsRandom': [54, 93, 10, 36, 34, 54, 34, 28]},
 {'Block': 1,
  'LocationNumber': 4,
  'StartingPositionRandom': 13,
  'TargetLocationsRandom': [73, 25, 40, 71, 91, 93, 83, 73]},
 {'Block': 1,
  'LocationNumber': 5,
  'StartingPositionRandom': 26,
  'TargetLocationsRandom': [53, 101, 11, 62, 24, 34, 43, 15]},
 {'Block': 1,
  'LocationNumber': 6,
  'StartingPositionRandom': 6,
  'TargetLocationsRandom': [66, 101, 67, 104, 100, 44, 74, 12]},
 {'Block': 1,
  'LocationNumber': 7,
  'StartingPositionRandom': 11,
  'TargetLocationsRandom': [59, 34, 34, 42, 100, 9, 59, 44]},
 {'Block': 1,
  'LocationNumber': 8,
  'StartingPositionRandom': 23,
  'TargetLoca

In [8]:
normalized = pd.json_normalize(subjectdf["PointingTaskRandomised"][0])
normalized.head()

,Block,LocationNumber,StartingPositionRandom,TargetLocationsRandom
0,1,1,4,"[28, 43, 99, 99, 107, 22, 92, 41]"
1,1,2,25,"[9, 85, 27, 32, 25, 31, 14, 2]"
2,1,3,27,"[54, 93, 10, 36, 34, 54, 34, 28]"
3,1,4,13,"[73, 25, 40, 71, 91, 93, 83, 73]"
4,1,5,26,"[53, 101, 11, 62, 24, 34, 43, 15]"


In [9]:
Final = pd.DataFrame(normalized.explode('TargetLocationsRandom'))
Final.head()

,Block,LocationNumber,StartingPositionRandom,TargetLocationsRandom
0,1,1,4,28
0,1,1,4,43
0,1,1,4,99
0,1,1,4,99
0,1,1,4,107


In [10]:
pd.json_normalize(subjectdf.PointingTaskStartingLocations")

SyntaxError: EOL while scanning string literal (<ipython-input-10-3246c7ab22f1>, line 1)

In [ ]:
Dataset = pd.DataFrame()

In [ ]:
%%timeit -r 4 -n 1000
StartingPointsnorm = pd.json_normalize(subjectdf["InitialOrientation"]).add_suffix('_InitialOrientation')
StartingPointsnorm.head()